### Loading Libraries

In [1]:
import os
import pandas as pd
import cv2
from matplotlib import pyplot as plt
from PIL import Image
from io import BytesIO
import numpy as np
import anndata
import joblib
from matplotlib import pyplot as plt
import scipy as sp
import seaborn as sns
from sklearn.cluster import AgglomerativeClustering
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.multioutput import MultiOutputClassifier, MultiOutputRegressor
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing import image as image_fun
import warnings
warnings.filterwarnings("ignore")

2022-07-05 18:05:50.745403: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/uqomulay/.conda/envs/Classification_STimage/lib/python3.8/site-packages/cv2/../../lib64:/opt/ohpc/pub/mpi/openmpi3-gnu8/3.1.4/lib:/opt/ohpc/pub/compiler/gcc/8.3.0/lib64
2022-07-05 18:05:50.745454: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### ResNet50 and Logistic Regression

In [2]:
#Computing ResNet50 features
def ResNet50_features(pre_model, anndata):
    resnet_features = []
    for imagePath in anndata.obs["tile_path"]:
        image = plt.imread(imagePath).astype('float32')
        image = np.expand_dims(image, axis=0)
        image = preprocess_input(image)
        resnet_features.append(pre_model.predict(image, batch_size=1))
        
    #Shape of resnet50 features is coming out as (no. of tiles, 1, no. of resnet features)
    resnet_features = np.asarray(resnet_features)
    anndata.obsm["resnet50_features"] = resnet_features.reshape(resnet_features.shape[0],resnet_features.shape[2])    
    
#Training Pre-Processing
def classification_preprocessing(anndata):
    gene_exp = anndata.to_df() 
    gene_exp['library_id'] = anndata.obs['library_id']
    gene_exp_zscore = gene_exp.groupby('library_id')[list(gene_exp.iloc[:,:-1].columns)].apply(lambda x: (x-x.mean())/(x.std()))
    anndata.obsm["true_gene_expression"] = pd.DataFrame(gene_exp_zscore.apply(lambda x: [0 if y <= 0 else 1 for y in x]),
                                                       columns = anndata.to_df().columns, index = anndata.obs.index)


#Logistic Regression Classifier
def LR_model(train_adata, iteration=10000, penalty_option="elasticnet", regularization_strength=0.1, optimization="saga", l1_l2_ratio=0.5):
    model_c = LogisticRegression(max_iter=iteration, penalty=penalty_option, C=regularization_strength,solver=optimization,l1_ratio=l1_l2_ratio)
    clf_resnet = MultiOutputClassifier(model_c).fit(train_adata.obsm["resnet50_features"], train_adata.obsm["true_gene_expression"])
    joblib.dump(clf_resnet, Path+'pickle/LRmodel.pkl')

### Performance Evaluation

In [32]:
def performance_metrics(test_adata, gene_list):
    AUROC = []; F1 = []; Precision = []; Recall = []
    for i in set(test_adata.obs["library_id"]):
        anndata_adata = test_adata[test_adata.obs["library_id"]==i]
        for j in gene_list:
            score_auroc = roc_auc_score(anndata_adata.obsm["predicted_gene_expression"][j],anndata_adata.obsm["true_gene_expression"][j])
            AUROC.append(score_auroc)

            score_f1 =  f1_score(anndata_adata.obsm["predicted_gene_expression"][j],anndata_adata.obsm["true_gene_expression"][j])
            F1.append(score_f1)

            score_precision =  precision_score(anndata_adata.obsm["predicted_gene_expression"][j],anndata_adata.obsm["true_gene_expression"][j])
            Precision.append(score_precision)

            score_recall =  recall_score(anndata_adata.obsm["predicted_gene_expression"][j],anndata_adata.obsm["true_gene_expression"][j])
            Recall.append(score_recall)

    Performance_metrics["Scores"] = pd.concat([pd.DataFrame(AUROC), pd.DataFrame(F1), pd.DataFrame(Precision), pd.DataFrame(Recall)])
    Performance_metrics['Patient'] = list(np.repeat(list(set(test_adata.obs["library_id"])),len(gene_list)))*4
    Performance_metrics['Genes'] = gene_list*len(set(Performance_metrics['Patient']))*4
    Performance_metrics['Metrics'] = ['AUROC']*len(AUROC)+['F1']*len(F1)+['Precision']*len(Precision)+['Recall']*len(Recall)

    sns.set(font_scale = 2, style="whitegrid",)
    plt.figure(figsize=(22.50,12.50))
    plt.ylim(-0.1, 1.10)
    im = sns.boxplot(x="Patient", y=0, hue="Metrics", data=Performance_metrics,linewidth=3.5,width=0.6)
    im.set_xticklabels(im.get_xticklabels(),rotation = 30)
    plt.legend(loc="lower right", frameon=True, fontsize=20)
    im.axhline(0.5, linewidth=2, color='r')

    sns.set(font_scale = 2, style="whitegrid",)
    plt.figure(figsize=(22.50,12.50))
    plt.ylim(-0.1, 1.10)
    im2 = sns.boxplot(x="Genes", y=0, hue="Metrics", data=Performance_metrics,linewidth=3.5,width=0.6)
    im2.set_xticklabels(im2.get_xticklabels(),rotation = 30)
    plt.legend(loc="lower right", frameon=True, fontsize=20)
    im2.axhline(0.5, linewidth=2, color='r')

    return im.figure.savefig(Path+'Classification_boxplot_cancer_immune_controls.png'), im2.figure.savefig(Path+'Classification_boxplot_cancer_immune_controls_genes.png')

### Cancer vs Non-Cancer Spot Clustering

In [35]:
# Clustering followed by Classification
def clustering(train_adata):
    clustering = AgglomerativeClustering(n_clusters = 2, affinity = 'euclidean', linkage = 'ward')
    model_c = LogisticRegression(max_iter=10000,penalty='elasticnet',C=0.1,solver='saga',l1_ratio=0.5)
    clf_can_v_non_can = model_c.fit(train_adata.obsm["true_gene_expression"],clustering.fit_predict(train_adata.obsm["true_gene_expression"]))
    joblib.dump(clf_can_v_non_can, Path+'pickle/resnet_block1_log_scaled_relu_clustering_logistic.pkl')

## LIME

In [5]:
def model_predict_gene(gene):
    i = gene_list_2.index(gene)
    def combine_model_predict(tile1):
        feature1 = resnet_model.predict(tile1)
        prediction = clf_resnet.predict_proba(feature1)
        return prediction[i]
    return combine_model_predict

def watershed_segment(image):
    annotation_hed = rgb2hed(image)
    annotation_h = annotation_hed[:,:,0]
    annotation_h *= 255.0 / np.percentile(annotation_h, q=0.01)
    thresh = skimage.filters.threshold_otsu(annotation_h)*0.9
    im_fgnd_mask = sp.ndimage.morphology.binary_fill_holes(
        annotation_h < thresh
    )
    distance = ndi.distance_transform_edt(im_fgnd_mask)
    coords = peak_local_max(distance, footprint=np.ones((5, 5)), labels=im_fgnd_mask)
    mask = np.zeros(distance.shape, dtype=bool)
    mask[tuple(coords.T)] = True
    markers, _ = ndi.label(mask)
    labels = watershed(annotation_h, markers, mask=im_fgnd_mask)
    im_nuclei_seg_mask = area_opening(labels, area_threshold=64).astype(np.int)
    map_dic = dict(zip(np.unique(im_nuclei_seg_mask), np.arange(len(np.unique(im_nuclei_seg_mask)))))
    im_nuclei_seg_mask = np.vectorize(map_dic.get)(im_nuclei_seg_mask)
    return im_nuclei_seg_mask

def LIME_heatmap(image_path, gene):
    image = np.asarray(image_fun.load_img(image))
    explanation = explainer.explain_instance(image, model_predict_gene(gene), top_labels=2, hide_color=0, num_samples=1000, segmentation_fn=watershed_segment)
    temp, mask = explanation.get_image_and_mask(1, positive_only=False, num_features=10000, hide_rest=True)
    dict_heatmap = dict(explanation.local_exp[1])
    heatmap = np.vectorize(dict_heatmap.get)(explanation.segments)
    return mask, heatmap

### User Input Data

In [6]:
Path = "/home/uqomulay/90days/STimage_outputs/"
all_data = anndata.read_h5ad(Path+"all_adata.h5ad")
gene_list = ['CD74', 'CD24', 'CD63', 'CD81', 'CD151', 
             'COX6C', 'TP53', 'PABPC1', 'GNAS', 'B2M', 'SPARC', 'HSP90AB1', 'TFF3', 'ATP1A1', 'FASN',
             'CCSER2', 'SYMPK', 'ANKRD17','PUM1']
all_data = all_data[:,gene_list]
all_data.obs["tile_path"] = Path+"tiles/tiles/"+all_data.obs["tile_path"].str.split('/', expand=True)[6]
model = ResNet50(weights='imagenet', pooling="avg", include_top = False)


iteration = 10000
penalty_option = 'elasticnet'
regularization_strength = 0.1
optimization = 'saga'
l1_l2_ratio = 0.5

train_data_library_id = ["block1"]#,"1142243F","CID4290","CID4465","CID44971","CID4535","block2"
train_adata = all_data[all_data.obs["library_id"].isin(["block1"])]
test_adata = all_data[all_data.obs["library_id"].isin(["block2","FFPE"])]

2022-07-05 18:06:19.455170: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/uqomulay/.conda/envs/Classification_STimage/lib/python3.8/site-packages/cv2/../../lib64:/opt/ohpc/pub/mpi/openmpi3-gnu8/3.1.4/lib:/opt/ohpc/pub/compiler/gcc/8.3.0/lib64
2022-07-05 18:06:19.455231: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-05 18:06:19.455262: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (delta089): /proc/driver/nvidia/version does not exist
2022-07-05 18:06:19.455556: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable the

In [19]:
#ResNet50_features(model, train_adata)
ResNet50_features(model, test_adata)
print("Done-1")

#classification_preprocessing(train_adata)
classification_preprocessing(test_adata)
print("Done-2")

#LR_model(train_adata)
print("Done-3")

1/1 [==============================] - 0s 94ms/step


1/1 [==============================] - 0s 75ms/step


1/1 [==============================] - 0s 83ms/step


1/1 [==============================] - 0s 75ms/step


1/1 [==============================] - 0s 74ms/step


...

1/1 [==============================] - 0s 71ms/step


1/1 [==============================] - 0s 87ms/step


1/1 [==============================] - 0s 79ms/step


1/1 [==============================] - 0s 71ms/step


1/1 [==============================] - 0s 76ms/step
Done-1
Done-2
Done-3


In [ ]:
clf_resnet = joblib.load(Path+'pickle/LRmodel.pkl')
test_adata.obsm["predicted_gene_expression"] = pd.DataFrame(clf_resnet.predict(test_adata.obsm["resnet50_features"]),columns=test_adata.to_df().columns,index=test_adata.obs.index)

performance_metrics(test_adata)

clustering(train_adata)
clf_can_v_non_can = joblib.load(Path+'pickle/resnet_block1_log_scaled_relu_clustering_logistic.pkl')
can_v_non_can_spot = pd.DataFrame(clf_can_v_non_can.predict(test_adata.obsm["predicted_gene_expression"]),index=test_adata.obs.index)
test_adata.obsm["clusters_can_v_non_can"] = can_v_non_can_spot

### Performance Evaluation with exceptional handling

In [ ]:
#Boxplot of Performance Metrics
def performance_metrics_exceptional(Path, gene_exp_binary_all_true, performance_metrics_all_predictions, gene_list):
    AUROC_genes = []; F1_genes = []; Precision_genes = []; Recall_genes = [];
    for dataset in range(len(gene_exp_binary_all_true)):
        for gene in range(len(gene_list)):
            try:
                score =  roc_auc_score(performance_metrics_all_predictions[dataset].iloc[:,gene],gene_exp_binary_all_true[dataset].iloc[:,gene])
                AUROC_genes.append(score)
            except:
                AUROC_genes.append(score)

    for dataset in range(len(gene_exp_binary_all_true)):
        for gene in range(len(gene_list)):
            try:
                score =  f1_score(performance_metrics_all_predictions[dataset].iloc[:,gene],gene_exp_binary_all_true[dataset].iloc[:,gene])
                F1_genes.append(score)
            except:
                F1_genes.append(score)

    for dataset in range(len(gene_exp_binary_all_true)):
        for gene in range(len(gene_list)):
            try:
                score =  precision_score(performance_metrics_all_predictions[dataset].iloc[:,gene],gene_exp_binary_all_true[dataset].iloc[:,gene])
                Precision_genes.append(score)
            except:
                Precision_genes.append(score)

    for dataset in range(len(gene_exp_binary_all_true)):
        for gene in range(len(gene_list)):
            try:
                score =  recall_score(performance_metrics_all_predictions[dataset].iloc[:,gene],gene_exp_binary_all_true[dataset].iloc[:,gene])
                Recall_genes.append(score)
            except:
                Recall_genes.append(score)

    AUROC_genes = pd.DataFrame(AUROC_genes); F1_genes = pd.DataFrame(F1_genes); Precision_genes = pd.DataFrame(Precision_genes); Recall_genes = pd.DataFrame(Recall_genes)
    Performance_metrics = pd.concat([AUROC_genes,F1_genes,Precision_genes,Recall_genes])
    Performance_metrics['patient'] = list(np.repeat(pd.concat(gene_exp_binary_all_true).loc[~(pd.concat(gene_exp_binary_all_true)['dataset'].isin(train_data_library_id))].drop_duplicates('dataset', keep='first')['dataset'].to_list(),len(gene_list)))*4
    Performance_metrics['genes'] = gene_list*len(set(Performance_metrics['patient']))*4
    Performance_metrics['measure'] = ['AUROC']*len(AUROC_genes)+['F1']*len(F1_genes)+['Precision']*len(Precision_genes)+['Recall']*len(Recall_genes)

    plt.figure(figsize=(19.20,10.80))
    im = sns.boxplot(x="patient", y=0, hue="measure", data=Performance_metrics,linewidth=4)
    im.axhline(0.5, linewidth=2, color='r')
    return im.figure.savefig(Path+'Classification_boxplot.png')